In [ ]:
conda create -n env_pytorch python=3.6

In [38]:
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util
import string
nltk.download('stopwords')
import pandas as pd
from tqdm.notebook import tqdm
from krovetzstemmer import Stemmer 
stemmer = Stemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anukritigan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import torch

In [39]:
ocassions_map  = pd.read_csv('groups_ocassion.csv') 

In [40]:
ocassions_map

,tail,kmeans,head
0,sports,3.0,sport
1,sport,3.0,sport
2,sports wear,1.0,sport
3,sport wear,1.0,sport
4,club wear,1.0,party
5,casual,0.0,casual
6,casuals,0.0,casual
7,formal wear,0.0,formal
8,casual wear,0.0,casual
9,wedding,2.0,wedding


In [41]:
ocassions_map['tail'] = ocassions_map['tail'].str.replace('wear','').str.strip()

In [42]:
ocassions_map

,tail,kmeans,head
0,sports,3.0,sport
1,sport,3.0,sport
2,sports,1.0,sport
3,sport,1.0,sport
4,club,1.0,party
5,casual,0.0,casual
6,casuals,0.0,casual
7,formal,0.0,formal
8,casual,0.0,casual
9,wedding,2.0,wedding


In [43]:
ocassions_map = pd.concat([ocassions_map,pd.DataFrame({'tail':list(ocassions_map['head'].drop_duplicates()),'head':list(ocassions_map['head'].drop_duplicates())})])

In [44]:
ocassions_map = ocassions_map[['head','tail']].drop_duplicates()

In [45]:
ocassions_map

,head,tail
0,sport,sports
1,sport,sport
4,party,club
5,casual,casual
6,casual,casuals
7,formal,formal
9,wedding,wedding
11,wedding,weddings
12,indoor,indoor
13,outdoor,outdoor


In [46]:
print(dict(ocassions_map[['tail','head']].values))

{'sports': 'sport', 'sport': 'sport', 'club': 'party', 'casual': 'casual', 'casuals': 'casual', 'formal': 'formal', 'wedding': 'wedding', 'weddings': 'wedding', 'indoor': 'indoor', 'outdoor': 'outdoor', 'indoors': 'indoor', 'winter': 'winter', 'summer': 'cotton', 'track': 'track', 'tracking': 'track', 'festive': 'festive', 'party': 'party', 'festival': 'festive', 'christmas': 'festive', 'beach': 'beach', 'ethnic': 'ethnic', 'ethnics': 'ethnic', 'day': 'casual', 'home': 'casual', 'everyday': 'casual', 'western': 'western', 'office': 'office', 'evening': 'evening', 'work': 'office', 'diwali': 'festive', 'sleep': 'night', 'cotton': 'cotton', 'night': 'night'}


In [47]:
ocassions_map

,head,tail
0,sport,sports
1,sport,sport
4,party,club
5,casual,casual
6,casual,casuals
7,formal,formal
9,wedding,wedding
11,wedding,weddings
12,indoor,indoor
13,outdoor,outdoor


In [48]:
x = {'gender_maps' : {'man': 'men', 'men': 'men', 'mans': 'men', 'mens': 'men', "man's": 'men', "men's": 'men',
                            "mans'": 'men', "mens'": 'men', 'male': 'men', 'husband': 'men', 'boyfriend': 'men',
                            'boy friend': 'men', 'gents': 'men', 'gent': 'men',
                            'woman': 'women', 'womans': 'women', "woman's": 'women', "womans'": 'women',
                            'women': 'women', 'womens': 'women', "women's": 'women', "womens'": 'women',
                            'lady': 'women', 'ladies': 'women', 'ladie': 'women', "lady's": 'women', 'female': 'women',
                            'wife': 'women', 'girlfriend': 'women', 'girl friend': 'women',
                            'girl': 'girls', 'girls': 'girls', "girl's": 'girls', "girls'": 'girls', 'boy': 'boys',
                            'boys': 'boys', "boy's": 'boys', "boys'": 'boys', 'kid': 'kids', 'kids': 'kids',
                            "kid's": 'kids', "kids'": 'kids',"bacchon":'kids',
                            'unisex': 'unisex', 'friend': 'unisex', 'baby': 'infant', 'babies': 'infant',
                            'couple': 'couples'},
'ocassions_maps' : {'sports': 'sport', 'sport': 'sport', 'club': 'party', 'casual': 'casual', 'casuals': 'casual',
                               'formal': 'formal', 'wedding': 'wedding', 'weddings': 'wedding', 'indoor': 'indoor',
                               'outdoor': 'outdoor', 'indoors': 'indoor', 'winter': 'winter', 'summer': 'cotton',
                               'tracking': 'track', 'festive': 'festive', 'party': 'party',
                               'festival': 'festive', 'christmas': 'festive', 'beach': 'beach', 'ethnic': 'ethnic',
                               'ethnics': 'ethnic', 'day': 'casual', 'home': 'casual', 'everyday': 'casual',
                               'western': 'western', 'office': 'office', 'evening': 'evening', 'work': 'office',
                               'diwali': 'festive', 'sleep': 'night', 'cotton': 'cotton', 'night': 'night', 
                               'traditonal':'ethnic', 'daily':'casual'
                              },
'ocassions_reverse' : {'sport': ['sport'],
 'party': ['party'],
 'casual': ['casual', 'day', 'home', 'everyday', 'daily'],
 'formal': ['formal'],
 'wedding': ['wedding'],
 'indoor': ['indoor'],
 'outdoor': ['outdoor'],
 'winter': ['winter','wool'],
 'summer': ['summer','cotton'],
 'track': ['track'],
 'festive': ['festive','festival', 'christmas', 'diwali'],
 'beach': ['beach'],
 'ethnic': ['ethnic', 'traditonal'],
 'western': ['western'],
 'office': ['office','work'],
 'evening': ['evening'],
 'night': ['night']},
     
'stops':list(set(stopwords.words('english')) - {"after","under"}),
'block_list':['single']
}

import json
json.dump(x,open("hnt_support.json",'w'))

In [49]:
tmp = {}
for i,j in x['ocassions_maps'].items():
    if j in tmp:
        if i.rstrip('s') not in tmp[j]:
            tmp[j].append(i)
    else:
        tmp[j] = []

In [50]:
tmp

{'sport': ['sport'],
 'party': ['party'],
 'casual': ['casuals', 'day', 'home', 'everyday', 'daily'],
 'formal': [],
 'wedding': ['weddings'],
 'indoor': ['indoors'],
 'outdoor': [],
 'winter': [],
 'cotton': ['cotton'],
 'track': [],
 'festive': ['festival', 'christmas', 'diwali'],
 'beach': [],
 'ethnic': ['ethnics', 'traditonal'],
 'western': [],
 'office': ['work'],
 'evening': [],
 'night': ['night']}

In [62]:
DEVICE = "cpu" #if cuda.is_available() and INFERENCE_DEVICE != 'cpu' else "cpu"

class HeadNTailModel:
    def __init__(self, head_tail_path, support_path ,head_queries, model_path='multi-qa-MiniLM-L6-dot-v1'):
        support = json.load(open(support_path))
        self.gender_maps = support['gender_maps']
        self.ocassions_maps = support['ocassions_reverse']
        self.stops = set(support['stops'])
        self.encoder = SentenceTransformer(model_path, device=DEVICE)
        self.head_tail = pd.read_csv(head_tail_path)
        self.heads_all = list(set(self.head_tail['head']))
        self.all_heads = self.encoder.encode(self.heads_all)
        self.query_cluster_map = dict(self.head_tail[['tail','head']].values)
        self.all_cluster = list(self.query_cluster_map)
        self.cluster_all = self.encoder.encode(self.all_cluster)
        self.synonyms = dict(self.head_tail[['tail','head']].groupby('head',as_index=0).agg({'tail':list}).values)
        self.query_embedding = lambda x: self.encoder.encode(x, show_progress_bar=False)
        self.block_list = support['block_list']
        self.head_queries = pd.read_csv(head_queries)
        self.norm_queries = list(self.head_queries['normalized_query'].dropna())
        self.norm_sets = list(self.head_queries['normalized_query'].dropna().str.split(' ').apply(set))
        self.fact = 21
        
    def get_synonyms(self, heads_all, all_heads, query, topn, th):
        sets = set(query.split(' '))
        chars = query.replace(' ', '')
        if query in self.all_cluster:
            return [query]
        elif chars in self.all_cluster:
            return [chars]

        out = []
        cos_scores = util.cos_sim(self.query_embedding(query), all_heads)[0]
        top_results = torch.topk(cos_scores, k=topn)
        for i, j in zip(top_results.indices, top_results.values):
            if j >= th:
                tmp = heads_all[i]
                if set(tmp.split(' ')) == sets:
                    return [tmp]
                elif chars == tmp.replace(' ', ''):
                    return [tmp]
                else:
                    out.append(tmp)                    
        return out

    def unpopulate(self, query, ner):
        for i, att in ner.items():
            if i in ['brand', 'gender', 'material', 'model_name', 'style',  'size', 'capacity',
                     'memory','occasions']:
                for att_ in att:
                    query= ' '.join([i for i in query.split(' ') if i not in att or i in self.block_list])
            elif i in ['color']:
                query= ' '.join([i for i in query.split(' ') if i not in att+['color','coloured','colored','colour','and']])
                
        split = list(filter(lambda x: len(x) > 0, query.split(" ")))
        if len(split) == 0:
            return ''
        if split[0].lower() in self.stops:
            split.pop(0)
        if len(split) == 0:
            return ''
        if split[-1].lower() in self.stops:
            split.pop()
        query = " ".join(split)
        return query

    def populate(self, unpopulated, ner):
        query = ' '
        for i, att in ner.items():
            if i in ['material','occasions', 'style', 'color', 'size','brand']:
                for att_ in att:
                    if att_ not in self.block_list:
                        query += att_ + " "
            elif i == 'gender':
                for att_ in att:
                    query += self.gender_maps.get(att_, att_) + " "
                if self.gender_maps.get(att_, att_) == 'men':
                    unpopulated = unpopulated.replace('dress','wear').strip()
                    
        query += unpopulated + " "
        for i, att in ner.items():
            if i in ['capacity', 'memory','model_name']:
                for att_ in att:
                    query += att_ + " "
        return query.strip()

    def get_head(self,tmp,tmp_split):
        try:
            tmp = self.norm_queries[self.norm_sets.index(tmp_split)]
            return tmp
        except:
            return tmp
        
        
    def get_head_query(self, query: str, ner,th = 0.95,enable=True):
        rk = 1
        query_split = set(query.split())
        tmp_ = self.get_head(query,query_split)

        df = {'input_query': query , 'head_query':tmp_ , 'synonyms_list':[tmp_],'boost':[{'rank':rk,'boost':self.fact-rk,'value':tmp_}]}

        if not enable : return df

        unpopulated = self.unpopulate(query, ner)
        if unpopulated != '':
            y = self.get_synonyms(self.all_cluster, self.cluster_all, unpopulated, 5, th)
            if len(y) > 0:
                y = self.query_cluster_map[y[0]]
                syns = self.synonyms[y]
            else:
                y = unpopulated
                syns = []
            for j in ['combo','set','pack']:
                if j in query and j not in y:
                    y += ' '+ j
                    syns = [k+ ' '+j if j not in k else k for k in syns ]
                    break
            
            tmp = self.populate(y, ner)
            
            tmp_split = set(tmp.split())
            matcher = tmp != query
            
            if matcher and tmp not in df['synonyms_list']:
                tmp_ = self.get_head(tmp,tmp_split)
                df['head_query'] = tmp_
                df['synonyms_list'].append(tmp_)
                rk+=1
                df['boost'].append({'rank':rk,'boost':self.fact-rk,'value':tmp_})
                
            if matcher:
                if 'occasions' in ner:
                    ner_ = ner.copy()
                    for i in self.ocassions_maps.get(ner['occasions'][0].replace('wear','').strip(), ner['occasions']):
                        ner_['occasions'] = [i]
                        tmp_ = self.populate(unpopulated, ner_)
                        tmp_ = self.get_head(tmp_,set(tmp_.split()))
                        if tmp_ != query and tmp_ != tmp and tmp_ not in df['synonyms_list']:
                            df['synonyms_list'].append(tmp_)
                            rk+=1
                            df['boost'].append({'rank':rk,'boost':self.fact-rk,'value':tmp_})
                else:
                    tmp_ = self.populate(unpopulated, ner)

                    if set(tmp_.split()) != set(query.split()) and set(tmp_.split()) != set(tmp.split()) and tmp_ not in df['synonyms_list']:
                        df['synonyms_list'].append(tmp_)
                        rk+=1
                        df['boost'].append({'rank':rk,'boost':self.fact-rk,'value':tmp_})
        return df

In [63]:
# hnt.get_synonyms(hnt.all_cluster, hnt.cluster_all, 'h&m', 10, 0.90)

In [65]:
hnt = HeadNTailModel('head_tail_maps.csv','hnt_support.json','head_queries_norm.csv')

In [72]:

hnt.get_head_query('black jack and jones tshirt',{'color':['black']},enable=True)

{'input_query': 'black jack and jones tshirt',
 'head_query': 'black jack jones tshirt',
 'synonyms_list': ['black jack and jones tshirt', 'black jack jones tshirt'],
 'boost': [{'rank': 1, 'boost': 20, 'value': 'black jack and jones tshirt'},
  {'rank': 2, 'boost': 19, 'value': 'black jack jones tshirt'}]}

In [ ]:
# hnt.query_cluster_map['wificamera']

In [ ]:
data = pd.read_csv('data/head_queries_with_qps722.csv')

In [ ]:
data

In [ ]:
# data['query.1']
data.columns

In [ ]:
data = data[['query','normalized_query','% User Searching', 'Auto Suggest Term Click', 'C. R. %',
       'Conversion Rate', 'Int. Searches (e427)', 'Int. Searches / Visits',
       'L1', 'L2', 'L3', 'L4', 'NER', 'Null Search %', 'Occurrences', 'Orders',
       'PV/V', 'Search CTR (New Def.)', 'Visits', 'brand', 'capacity', 'color',
       'gender', 'material', 'memory', 'message', 'model_name',
        'occasions', 'product_type',  'result_type',
       'searched_term', 'size', 'style', 'total_count']]

In [ ]:
data['NER'] = data['NER'].replace({'0':'{}'}).apply(eval)

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
def get_fields(df):
    x = hnt.get_head_query(df['query'],df['NER'])
    for i,j in x.items():
        df[i] = j
    return df

In [ ]:
data = data.progress_apply(get_fields,axis=1)

In [ ]:
data.columns

In [ ]:
data[['query', 'normalized_query', 'input_query', 'head_query', 'synonyms_list', 'boost','% User Searching',
       'Auto Suggest Term Click', 'C. R. %', 'Conversion Rate',
       'Int. Searches (e427)', 'Int. Searches / Visits', 'L1', 'L2', 'L3',
       'L4', 'NER', 'Null Search %', 'Occurrences', 'Orders', 'PV/V',
       'Search CTR (New Def.)', 'Visits', 'brand', 'capacity', 'color',
       'gender', 'material', 'memory', 'message', 'model_name', 'occasions',
       'product_type', 'result_type', 'searched_term', 'size', 'style',
       'total_count']].to_csv('tail_share_v4_8922.csv',index=0)

In [ ]:
data

In [ ]:
data

In [ ]:
brands

In [ ]:
final_data = pd.read_csv('head_tail_maps.csv')

In [ ]:
final_data['head'] = final_data['head'].str.replace('omen','').str.strip()

In [ ]:
# final_data

In [ ]:
final_data = final_data[~final_data['tail'].str.contains('year|size')]

In [ ]:
final_data = final_data[['tail','head']].drop_duplicates(subset=['tail'])

In [ ]:
indx = {j:i for i,j in enumerate(final_data['head'].drop_duplicates())}

In [ ]:
final_data['index'] = final_data['head'].apply(lambda x: indx[x] )

In [ ]:
final_data.to_csv('head_tail_maps.csv',index=0)

In [ ]:
brand = [i for i in brands if len(i.split())> 1]

In [ ]:
final_data = pd.concat([final_data,pd.DataFrame({'tail':brand,'head':brand})])

In [ ]:
final_data = pd.concat([final_data,pd.DataFrame({'tail':list(mnumber['model_name'].drop_duplicates()),'head':list(mnumber['model_name'].drop_duplicates())})])

In [ ]:
final_data

In [ ]:
head = pd.read_csv('data/head_queries_with_qps722.csv')
pptype = pd.read_csv('data/product_type_with_qps722.csv')
mnumber = pd.read_csv('data/modelnames0718.csv')
tail = pd.read_csv('data/tail_queries_with_ner.csv')


In [ ]:
def eval_data(head):
    cols = ['brand','gender','occasions','product_type','material','model_name','style','color','size','capacity','memory','NER']
    head = head.fillna('[]')
    for i in cols:
#         print(i)
        if i in head.columns:
            head[i] = head[i].apply(eval)
    return head

In [ ]:
pptype = eval_data(pptype)
head = eval_data(head)
tail = eval_data(tail)

In [ ]:
pptype['unpopulated'] = pptype.apply(lambda x: hnt.unpopulate(x['normalized_query'],x['NER']),axis=1)
head['unpopulated'] = head.apply(lambda x: hnt.unpopulate(x['normalized_query'],x['NER']),axis=1)
tail['unpopulated'] = tail.apply(lambda x: hnt.unpopulate(x['normalized_query'],x['NER']),axis=1)

In [ ]:
ocassions = pd.concat([pptype['occasions'].explode().dropna().drop_duplicates(),head['occasions'].explode().dropna().drop_duplicates(),tail['occasions'].explode().dropna().drop_duplicates()]).drop_duplicates()
brands = pd.concat([pptype['brand'].explode().dropna().drop_duplicates(),head['brand'].explode().dropna().drop_duplicates(),tail['brand'].explode().dropna().drop_duplicates()]).drop_duplicates()

In [ ]:
final_data = pd.read_csv('head_tail_maps.csv')

In [ ]:
final_data = final_data[~final_data['tail'].str.contains('1|2|3|4|5|6|7|8|9|0')]

In [ ]:
final_data['tail'] = final_data['tail'].apply(lambda x: ' '.join([i for i in x.split() if len(i) >2 ]).strip(' '))
final_data['head'] = final_data['head'].apply(lambda x: ' '.join([i for i in x.split() if len(i) >2 ]).strip(' '))

In [ ]:
final_data = pd.read_csv('head_tail_ner.csv')

In [ ]:
final_data = final_data.drop_duplicates()

In [ ]:
final_data['brand'] = final_data['brand'].apply(eval)

In [ ]:
brand_to_igonre = set(final_data['brand'].explode().dropna().drop_duplicates()) - {'and'}

In [ ]:
final_data = final_data.explode('brand')
final_data = final_data[~final_data.brand.isin(brand_to_igonre)]

In [ ]:
final_data[['tail','head','index']].to_csv('head_tail_semi.csv',index=0)

In [ ]:
# pptype[pptype.unpopulated.str.isalnum()]
# mnumber

In [ ]:
ocassions.to_csv('ocassions_data.csv',index=0)

In [ ]:
pptype['unpopulated'] = pptype.unpopulated.str.replace('1|2|3|4|5|6|7|8|9|0','')
pptype[pptype.unpopulated.str.contains('1|2|3|4|5|6|7|8|9|0')].tail(20)
pptype = pptype.drop_duplicates('unpopulated')
pptype['unpopulated'] = pptype.unpopulated.apply(lambda x: ' '.join([i for i in x.split() if len(i) >2 or i not in ['yrs','year','years','utsav']]).strip(' '))

In [ ]:
# head['Search CTR (New Def.)'].apply(str).str.replace('\[\]','0').fillna(0).astype(float)

In [ ]:
head['score'] = head['Search CTR (New Def.)'].apply(str).str.replace('\[\]','0').fillna(0).astype(float) * head['Occurrences'].apply(str).str.replace('\[\]','0').fillna(0).astype(float)
head['score'] = head['score'].fillna(0)

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
head = head.sort_values(by ='score',ascending=False)
head = head[head.unpopulated.apply(len) != 0]

In [ ]:
head['unpopulated'] = head.unpopulated.str.replace('1|2|3|4|5|6|7|8|9|0','')
head[head.unpopulated.str.contains('1|2|3|4|5|6|7|8|9|0')].tail(20)
head = head.drop_duplicates('unpopulated')
head['unpopulated'] = head.unpopulated.apply(lambda x: ' '.join([i for i in x.split() if len(i) >2 and i not in ['yrs','year','years','utsav']]).strip(' '))

In [ ]:
head

In [ ]:
tail['unpopulated'] = tail.unpopulated.str.replace('1|2|3|4|5|6|7|8|9|0','')
tail[tail.unpopulated.str.contains('1|2|3|4|5|6|7|8|9|0')].tail(20)
tail = tail.drop_duplicates('unpopulated')
tail['unpopulated'] = tail.unpopulated.apply(lambda x: ' '.join([i for i in x.split() if len(i) >2 or i not in ['yrs','year','years','utsav']]).strip(' '))

In [ ]:
final_data[final_data['tail'].str.contains('n ')]

In [ ]:
final_data.to_csv('head_tail_map_draft.csv',index=0)

In [ ]:
# final_data[final_data['index'].isna()]

In [ ]:
trmp = []
for i in list(brands):
    tmp = final_data[(final_data['tail'].str.startswith(i+' ')) | (final_data['tail'].str.endswith(' '+i)) | (final_data['tail'].str.contains(' '+i+' ')) ]
    if len(tmp) !=0:
        trmp.append(i)
        print(i,len(tmp))

In [ ]:
prods = list(pd.concat([head['unpopulated'], pptype['unpopulated']]).drop_duplicates())

In [ ]:
import hdbscan
def get_clusters(prods,eps=[0.002,0.004,0.02,0.04],min_size = [2,3,4]):
    groups = pd.DataFrame({'query':prods})
    all_heads = hnt.encoder.encode(prods)
    for i in tqdm(min_size):
        for j in tqdm(eps):
            clusterer = hdbscan.HDBSCAN( min_cluster_size=i,cluster_selection_epsilon=j)
            clusterer.fit(all_heads)
            groups['cluster_'+str(i)+'_'+str(j)] = clusterer.labels_
    return clusterer , groups

In [ ]:
clusterer , groups = get_clusters(list(ocassions),[0.002,0.004,0.1], [2,3,4])

In [ ]:
all_heads = hnt.encoder.encode(list(ocassions))

In [ ]:
max(groups['cluster_2_0.002'])

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=0).fit(all_heads)
# kmeans.predict([[0, 0], [12, 3]])


In [ ]:
groups['kmeans']  =kmeans.labels_


In [ ]:
groups

In [ ]:
groups.to_csv('groups_ocassion.csv',index=0)

In [ ]:
mapper = pd.read_csv('head_tail_maps.csv')

In [ ]:
# maaper = pd.read_csv('head_tail_map_v2.csv')
# maaper2 = pd.read_csv('head_maps1.csv')

In [ ]:
mapper

In [ ]:
groups